In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import re
from scipy.sparse import csr_matrix

In [2]:
# import data
game_data_clean = pd.read_csv("game_data_clean.csv")
game_data_clean.head()

,Unnamed: 0,titles,released,developers,publishers,genres
0,0,101 Dalmatians: Escape from DeVil Manor,1999,DreamForge Intertainment,Disney Interactive,"Puzzle, Strategy, Adventure"
1,1,102 Dalmatians: Puppies to the Rescue,2000,Crystal Dynamics,Eidos Interactive,"Platform, Adventure"
2,2,140,2016,Double Fine Productions,Abstraction Games,"Music, Platform, Puzzle, Strategy, Indie"
3,3,1001 Spikes,2014,8bits Fanatics,Nicalis,"Platform, Adventure, Indie"
4,4,The 11th Hour,1995,Trilobyte,Virgin Interactive,"Point-and-click, Puzzle, Adventure"


In [3]:
game_data_clean.rename(columns={'Unnamed: 0':'gameid'}, inplace=True)

In [4]:
game_data_clean.head()

,gameid,titles,released,developers,publishers,genres
0,0,101 Dalmatians: Escape from DeVil Manor,1999,DreamForge Intertainment,Disney Interactive,"Puzzle, Strategy, Adventure"
1,1,102 Dalmatians: Puppies to the Rescue,2000,Crystal Dynamics,Eidos Interactive,"Platform, Adventure"
2,2,140,2016,Double Fine Productions,Abstraction Games,"Music, Platform, Puzzle, Strategy, Indie"
3,3,1001 Spikes,2014,8bits Fanatics,Nicalis,"Platform, Adventure, Indie"
4,4,The 11th Hour,1995,Trilobyte,Virgin Interactive,"Point-and-click, Puzzle, Adventure"


In [5]:
game_data_clean.dtypes

gameid         int64
titles        object
released      object
developers    object
publishers    object
genres        object
dtype: object

In [6]:
#verify no null data
game_data_clean.isnull().sum()

gameid        0
titles        0
released      0
developers    0
publishers    0
genres        0
dtype: int64

In [7]:
#simplify variable
game_data = game_data_clean

In [8]:
#genres as categories
game_data["genres"].str.get_dummies(sep=',')

,"""Hack and slash/Beat em up""",Adventure,Arcade,Card & Board Game,Indie,MOBA,Music,Pinball,Platform,Puzzle,...,Racing,Real Time Strategy (RTS),Role-playing (RPG),Shooter,Simulator,Sport,Strategy,Tactical,Turn-based strategy (TBS),Visual Novel
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3743,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3744,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3745,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3746,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [9]:
game_metadata = pd.concat([pd.get_dummies(game_data_clean["developers"]),
                          pd.get_dummies(game_data_clean["publishers"]), 
                          game_data_clean["genres"].str.get_dummies(',')], axis = 1)

game_data["titles"] = game_data["titles"].map(lambda titles:re.sub('[^A-Za-z0-9]+', " ", titles))

In [10]:
game_metadata.head()

,10tacle Studios Belgium,10tons Entertainment,11 Bit Studios,14 Degrees East,1C Company,1C:Ino-Co,1C:Maddox Games,2 by 3 Games,"2015, Inc.",21-6 Productions,...,Racing,Real Time Strategy (RTS),Role-playing (RPG),Shooter,Simulator,Sport,Strategy,Tactical,Turn-based strategy (TBS),Visual Novel
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
from sklearn.preprocessing import MinMaxScaler

In [12]:
scaler = MinMaxScaler()
game_metadata = scaler.fit_transform(game_metadata)

In [13]:
np.round(game_metadata,2)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [61]:
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(n_neighbors=20, algorithm='ball_tree').fit(game_metadata)

In [63]:
distances, indices = knn.kneighbors(game_metadata)

In [18]:
def get_index_from_titles(title):
    return game_data[game_data["titles"]==title].index.tolist()[0]

In [16]:
all_game_titles = list(game_data.titles.values)

In [57]:
def print_recs(query = None, id = None):
    if id:
        for id in indices[id][1:]:
            print(game_data["titles"][id])
    if query:
        found_id = get_index_from_titles(query)
        for id in indices[found_id][1:]:
            print(game_data["titles"][id])

In [54]:
print_recs(id=4)

The 7th Guest
Danger on Deception Island
Danger By Design
Broken Sword Shadow of the Templars The Directors Cut
7 Sins
Byzantine The Betrayal
Broken Sword II The Smoking Mirror
Broken Sword The Shadow of the Templars
The Curse of Monkey Island


In [55]:
all_game_titles

['101 Dalmatians Escape from DeVil Manor',
 '102 Dalmatians Puppies to the Rescue',
 '140',
 '1001 Spikes',
 'The 11th Hour',
 '12Riven The Psi Climinal of Integral',
 '1914 Shells of Fury',
 '2 on 2 Open Ice Challenge',
 '2002 FIFA World Cup',
 '2006 FIFA World Cup',
 '25 To Life',
 '3D Ultra Minigolf',
 '3 D Ultra Pinball',
 '3D Body Adventure',
 '428 Shibuya Scramble',
 '4x4 EVO 2',
 '4x4 Evolution',
 '5',
 '6180 the moon',
 '7 Sins',
 '7 Wonders of the Ancient World',
 'The 7th Guest',
 '80 Days',
 '9 The Last Resort',
 'A Game of Thrones Genesis',
 'A New Beginning',
 'A Virus Named TOM',
 'A2 Racer',
 'ABC Sports Indy Racing',
 'Abomination The Nemesis Project',
 'Ace Online',
 'Ace Ventura',
 'Aces of the Galaxy',
 'Achtung Spitfire ',
 'Act of War Direct Action',
 'Act of War High Treason',
 'Activision Anthology',
 'Actua Ice Hockey',
 'Actua Pool',
 'Addiction Pinball',
 'Adrift',
 'Advent Rising',
 'Adventure Pinball Forgotten Island',
 'The Adventures of Fatman',
 'The Adve

In [72]:
print_recs("Diablo II")

Diablo III
Diablo II Lord of Destruction
Diablo
Phantasy Star Online
Silverfall
Castlevania Lords of Shadow
Torchlight
Final Fantasy XV Windows Edition
Diablo Hellfire
Knights of the Temple II
Dark Souls Remastered
Star Wars The Force Unleashed
Blaze and Blade Eternal Quest
Dark Souls Prepare to Die Edition
Assassins Creed Origins
Dark Sun Online Crimson Sands
Arthurs Knights
Arx Fatalis
Assassins Creed Valhalla
